
# Look at association of genetic status @ session 1 B
Changes to previous version:
- do marginal FDR correction
- put swarmplots on the boxplots
- have only one plot per subtype

In [1]:
%matplotlib notebook

In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import nilearn as nil
import seaborn as sns
import nibabel as nib
from scipy import io as sio
import statsmodels.api as sm
from nilearn import plotting as nlp
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
from statsmodels.sandbox.stats import multicomp as smi

In [3]:
# Paths
model_path = '/data1/guillimin/sim_data/simons_vip/pheno/pheno_tiv2.csv'
#weights_path = '/data1/guillimin/sim_data/simons_vip/subtype/subtypes3_16p_session1/subtype_weights.mat'
weights_path = '/data1/guillimin/sim_data/simons_vip/svip_subtype_TIV/subtype_weights.mat'

#sbt_map_temp= '/data1/guillimin/sim_data/simons_vip/subtype/subtypes3_16p_session1/network_{}/mean_subtype.nii.gz'
sbt_map_temp= '/data1/guillimin/sim_data/simons_vip/svip_subtype_TIV/network_{}/mean_subtype.nii.gz'

In [4]:
model = pd.read_csv(model_path, delimiter=';')

In [5]:
model.head()

,Unnamed: 0,frames_scrubbed.,frames_OK,FD,FD_scrubbed,session,sex,family,age_zscore,age_months,...,suspected_diagnosis1,suspected_diagnosis2,ID2,GM,WM,CSF,TBV,TIV,Genetic_status.1,ID
0,s14702xx3xFCAP1_session1_rest,39,79,0.305145,0.227978,session1,1,14702,1.687134,95,...,315-32-mixed-receptive-expressive-language-dis...,NaN,s14702xx3xFCAP1,0.964216,0.396450,0.262746,1.360666,1.623412,1,14702x3
1,s14714xx18xFCAP1_session1_rest,8,110,0.129200,0.118603,session1,1,14714,-0.208128,249,...,NaN,NaN,s14714xx18xFCAP1,0.740053,0.415315,0.393928,1.155368,1.549296,1,14714x18
2,s14725xx48xFCAP1_session1_rest,35,83,0.342359,0.249759,session1,1,14725,0.335883,415,...,309-0-adjustment-disorder-with-depressed-mood,311-depressive-disorder-nos,s14725xx48xFCAP1,0.661548,0.374882,0.226072,1.036430,1.262502,3,14725x48
3,s14729xx5xFCAP1_session1_rest,35,83,0.228533,0.158575,session1,1,14729,1.757329,99,...,296-90-mood-disorder-nos,NaN,s14729xx5xFCAP1,0.839764,0.380459,0.637733,1.220223,1.857956,1,14729x5
4,s14732xx6xFCAP1_session1_rest,34,84,0.199322,0.144287,session1,2,14732,-1.138210,131,...,NaN,315-32-mixed-receptive-expressive-language-dis...,s14732xx6xFCAP1,0.687705,0.231177,0.330932,0.918882,1.249814,1,14732x6


In [6]:
tmp = sio.loadmat(weights_path)
weights = tmp['weight_mat']

In [7]:
n_sbt, n_net = weights.shape[1:]

In [9]:
p_mat = np.zeros((n_sbt, n_net))
res_list = list()
for nid in range(n_net):
    tmp_list = list()
    for sid in range(n_sbt):
        m = sm.OLS(weights[:, sid, nid], model[['g1','g2','g3']])
        res = m.fit()
        p_mat[sid, nid] = res.f_pvalue
        tmp_list.append(res)
    res_list.append(tmp_list)

In [10]:
pcorr_vec = smi.multipletests(p_mat.flatten(), alpha=0.05, method='fdr_bh')
pcorr_mat = np.reshape(pcorr_vec[0], (n_sbt, n_net))

In [16]:
p_mat

array([[ 0.21914223,  0.27073091,  0.5275454 ,  0.17705027,  0.51800223,
         0.80826912,  0.01375848,  0.20454397,  0.10373806,  0.42615782,
         0.07411997,  0.71526929],
       [ 0.35862254,  0.08056252,  0.27237562,  0.33228972,  0.20682879,
         0.91994755,  0.34261553,  0.38535989,  0.11668905,  0.06743058,
         0.51759543,  0.17159479],
       [ 0.22489061,  0.45489077,  0.0820101 ,  0.16209854,  0.14912332,
         0.87607411,  0.35739121,  0.05162909,  0.55965077,  0.01848546,
         0.64032927,  0.53664192]])

In [11]:
pcorr_mat

array([[False, False, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False]], dtype=bool)

In [12]:
# Run post hoc tests on the subtypes that show effects
hits = np.argwhere(pcorr_mat)
n_hits = len(hits)

In [13]:
# Sort hits by network
ind = np.argsort([x[1] for x in hits])
hits = hits[ind]

In [14]:
# Define the post hocs to run
post_hocs = 'g1 = g2, g2 = g3, g1 = g3'
post_dict = {0:'g1vg2', 1:'g2vg3', 2:'g1vg3'}
hit_dict = dict()

#for hit_sbt, hit_net in hits:
for hit_sbt, hit_net in hits:
    if not hit_net in hit_dict.keys():
        hit_dict[hit_net] = dict()
    res = res_list[hit_net][hit_sbt]
    pvals = res.t_test(post_hocs).pvalue
    # FDR
    corr = smi.multipletests(pvals, alpha=0.05, method='fdr_bh')
    # Store
    hit_dict[hit_net][hit_sbt] = corr

- G1 - Dels (Autism)
- G2 - Controls
- G3 - Dups (Schizo)

In [15]:
for net in sorted(hit_dict.keys()):
    tmpd = hit_dict[net]
    n_plots = len(tmpd.keys())
    # Prepare figure
    fig = plt.figure(figsize=(12,4*n_plots))
    gs = gridspec.GridSpec(n_plots, 3, hspace=0.4)
    
    for pid, sbt in enumerate(sorted(tmpd.keys())):
        # Define axis
        ax = fig.add_subplot(gs[pid,0])
        fd_pass, pval, a, b = tmpd[sbt]
        n_tests = np.sum(fd_pass)
        
        # Plot the main plot
        sns.boxplot(model.genetic, weights[:, sbt, net], ax=ax)
        # Add swarm plot
        sns.boxplot(model.genetic, weights[:, sbt, net], ax=ax)
        sns.swarmplot(model.genetic, weights[:, sbt, net], ax=ax, size=4, linewidth=1)
        # Compute text overlay
        y = weights[:, sbt, net].max()
        h = y*0.15*n_tests
        y += y*0.1
        
        # Go through the significant tests
        for tid, test in enumerate(np.argwhere(fd_pass)):
            # figure out the test
            if test[0] == 0:
                x1, x2 = 0, 1
            elif test[0] == 1:
                x1, x2 = 1, 2
            elif test[0] == 2:
                x1, x2 = 0, 2
            
            ax.plot([x1, x1, x2, x2], [y, y+h*(tid+1), y+h*(tid+1), y], lw=1.5, c='k')
            ax.text((x1+x2)*.5, y+h*(tid+1), 'p={:.3f}'.format(pval[test][0]), ha='center', va='bottom', color='k')
            
        # Set axis
        (dn, up) = ax.get_ylim()
        ax.set_ylim((dn, up+h*n_tests))
        #ax.set_ylim((-0.8, 1.3))
        ax.set_title('Network {} Subtype {}'.format(net+1, sbt+1))
        ax.set_xticklabels(['deletion', 'control', 'duplication'])
        ax.set_ylabel('subtype weights')
        
        # Add the network plot
        ax2 = fig.add_subplot(gs[pid,1:])
        tmp_i = nib.load(sbt_map_temp.format(net+1))
        # Load the corresponding network
        nlp.plot_stat_map(nil.image.index_img(tmp_i, sbt), display_mode='x', threshold=0, 
                          cut_coords=[-50, -30, -10, 10, 30, 50], black_bg=True,axes=ax2,
                          title='Net {} Sbt {}'.format(net+1, sbt+1))